In [ ]:
import requests
from bs4 import BeautifulSoup
import os
import re

BASE_URL = "https://savings.gov.pk/rs-750-prize-bond-draw/"
DOWNLOAD_DIR = "results"
os.makedirs(DOWNLOAD_DIR, exist_ok=True)

def download_txt_files():
    response = requests.get(BASE_URL)
    soup = BeautifulSoup(response.text, "html.parser")

    # Find all <a> tags that point to .txt files
    links = soup.find_all("a", href=re.compile(r'\.txt$'))

    for link in links:
        file_url = link['href']
        file_name = file_url.split("/")[-1]
        save_path = os.path.join(DOWNLOAD_DIR, file_name)

        try:
            r = requests.get(file_url)
            r.raise_for_status()
            with open(save_path, 'wb') as f:
                f.write(r.content)
            print(f"✅ Downloaded: {file_name}")
        except Exception as e:
            print(f"❌ Failed to download {file_url}: {e}")

if __name__ == "__main__":
    download_txt_files()


## Match Bond Numbers

In [ ]:
                                                                                                                     
import os
import re

def load_bonds(file_path):
    with open(file_path, 'r') as file:
        return set(line.strip() for line in file if re.fullmatch(r'\d{6}', line.strip()))

def extract_bond_numbers(file_content):
    return set(re.findall(r'\b\d{6}\b', file_content))

def check_results(bond_file, results_dir='bond_results'):
    your_bonds = load_bonds(bond_file)
    winners = {}

    for file_name in os.listdir(results_dir):
        if not file_name.endswith(".txt"):
            continue
        
        file_path = os.path.join(results_dir, file_name)
        with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
            content = f.read()
        
        draw_numbers = extract_bond_numbers(content)
        matched = your_bonds & draw_numbers                                        
  
  
        
        if matched:
            winners[file_name] = matched

    return winners

# --- RUN ---
if __name__ == "__main__":
    result = check_results("my_bonds.txt")

    if result:
        print("🎉 Winning bonds found:\n")
        for draw_file, bonds in result.items():
            print(f"📄 {draw_file}:")
            for bond in sorted(bonds):
                print(f"   🏆 {bond}")
            print()
    else:
        print("🙃 No winning bonds found in any draw files.")


### For downloading doc file

In [ ]:
import os
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from pathlib import Path
from docx import Document

# Setup
BASE_URL = "https://savings.gov.pk/rs-750-prize-bond-draw/"
RESULTS_DIR = Path("results")
RESULTS_DIR.mkdir(exist_ok=True)

# Supported extensions
extensions = ['.txt', '.docx']  # .doc skipped for now

def convert_docx_to_txt(docx_path, txt_path):
    try:
        doc = Document(docx_path)
        with open(txt_path, 'w', encoding='utf-8') as f:
            for para in doc.paragraphs:
                f.write(para.text + '\n')
        print(f"✅ Converted: {docx_path.name} → {txt_path.name}")
    except Exception as e:
        print(f"❌ Failed to convert {docx_path.name}: {e}")

# Fetch and parse webpage
response = requests.get(BASE_URL)
soup = BeautifulSoup(response.text, 'html.parser')

for link in soup.find_all('a', href=True):
    file_url = urljoin(BASE_URL, link['href'])
    file_name = os.path.basename(link['href'])
    ext = Path(file_name).suffix.lower()

    if ext not in extensions and ext != '.doc':
        continue

    file_path = RESULTS_DIR / file_name
    txt_path = RESULTS_DIR / (Path(file_name).stem + ".txt")

    # Skip if .txt already exists
    if txt_path.exists():
        print(f"⚠️ Skipping (already converted): {txt_path.name}")
        continue

    # Skip if raw file is already downloaded (.txt only)
    if ext == '.txt' and file_path.exists():
        print(f"⚠️ Skipping (already downloaded): {file_path.name}")
        continue

    # Download file
    try:
        file_data = requests.get(file_url)
        with open(file_path, 'wb') as f:
            f.write(file_data.content)
        print(f"📥 Downloaded: {file_name}")

        # Convert if .docx
        if ext == '.docx':
            convert_docx_to_txt(file_path, txt_path)

        elif ext == '.doc':
            print(f"⚠️ Skipping .doc file (not supported yet): {file_name}")

    except Exception as e:
        print(f"❌ Error downloading or processing {file_name}: {e}")
